In [ ]:
from google.cloud import bigquery, pubsub_v1
from google.oauth2 import service_account
import requests, json, time, os

project_id = "persuasive-pipe-384604"
topic_name = 'stock_streaming'
subscription_name = "stock_subscriber"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\karth\Documents\key\gcpkey.json"

#Create subscriber client
bq_client = bigquery.Client(project=project_id)
subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_name)
    
# Set table ID and schema
    
dataset_id = "bigdata"
table_id = "streaming_stock"
# Construct the BigQuery table reference
table_ref = bq_client.dataset(dataset_id).table(table_id)
schema = [
    bigquery.SchemaField("symbol", "STRING"),
    bigquery.SchemaField("timestamp", "TIMESTAMP"),
    bigquery.SchemaField("open", "FLOAT"),
    bigquery.SchemaField("high", "FLOAT"),
    bigquery.SchemaField("low", "FLOAT"),
    bigquery.SchemaField("close", "FLOAT"),
    bigquery.SchemaField("volume", "INTEGER"),
]
    
def callback(message):
    
# Parse the message payload as JSON
    data = json.loads(message.data.decode('utf-8'))
    print(f"Received message: {data}")

# Insert the message into the BigQuery table
    errors = bq_client.insert_rows_json(table_ref, [data])
    if errors:
        print(f"Encountered errors while inserting rows: {errors}")
    else:
        print("Rows inserted successfully.")

# Acknowledge the message so it's removed from the subscription
    message.ack()

# Start the subscriber client and wait for messages
print("Starting subscriber...")
subscriber.subscribe(subscription_path, callback=callback)
